## 准备数据

In [37]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [38]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [39]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256]))  # 第一个全连接层的权重矩阵，输入大小为 784，输出大小为 128
        self.b1 = tf.Variable(tf.zeros([256]))               # 第一个全连接层的偏置向量，大小为 128
        self.W2 = tf.Variable(tf.random.normal([256, 128]))  # 第二个全连接层的权重矩阵，输入大小为 128，输出大小为 10
        self.b2 = tf.Variable(tf.zeros([128]))               # 第二个全连接层的偏置向量，大小为 10
        self.W3 = tf.Variable(tf.random.normal([128, 10]))   # 第二个全连接层的权重矩阵，输入大小为 128，输出大小为 10
        self.b3 = tf.Variable(tf.zeros([10]))                # 第二个全连接层的偏置向量，大小为 10
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28*28])  # 将输入 x 展平成大小为 (batch_size, 28*28) 的张量
        x = tf.matmul(x, self.W1) + self.b1  # 第一个全连接层，加上偏置项
        x = tf.nn.relu(x)                    # 使用 ReLU 激活函数
        x = tf.matmul(x, self.W2) + self.b2  # 第二个全连接层，加上偏置项
        x = tf.nn.relu(x)
        logits = tf.matmul(x, self.W3) + self.b3
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [40]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.W3, model.b1, model.b2, model.b3]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [41]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2027.955 ; accuracy 0.09481667
epoch 1 : loss 1030.053 ; accuracy 0.15226667
epoch 2 : loss 768.59753 ; accuracy 0.15981667
epoch 3 : loss 619.1888 ; accuracy 0.19991666
epoch 4 : loss 472.53177 ; accuracy 0.29031667
epoch 5 : loss 321.302 ; accuracy 0.32691666
epoch 6 : loss 261.53857 ; accuracy 0.38018334
epoch 7 : loss 211.50824 ; accuracy 0.40086666
epoch 8 : loss 177.8928 ; accuracy 0.44778332
epoch 9 : loss 156.77013 ; accuracy 0.47176668
epoch 10 : loss 144.18077 ; accuracy 0.49863333
epoch 11 : loss 134.30977 ; accuracy 0.51388335
epoch 12 : loss 126.15695 ; accuracy 0.5326167
epoch 13 : loss 119.18544 ; accuracy 0.54606664
epoch 14 : loss 113.17845 ; accuracy 0.56048334
epoch 15 : loss 107.9393 ; accuracy 0.5732
epoch 16 : loss 103.31987 ; accuracy 0.5847
epoch 17 : loss 99.20422 ; accuracy 0.59398335
epoch 18 : loss 95.49662 ; accuracy 0.60258335
epoch 19 : loss 92.13365 ; accuracy 0.6110167
epoch 20 : loss 89.07394 ; accuracy 0.6183
epoch 21 : loss 86.2768 ; a